In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?dl=0

--2023-06-14 07:02:25--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2023-06-14 07:02:26--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9f7f268526ae0b015cde1a709c.dl.dropboxusercontent.com/cd/0/inline/B9_rywAU_ogGIukQN1rdO3bB3SfEpTCnxV2z7FneG500DNRHDr7rEWnxSt-Lo3fXIoW6SHmrpP8Ccnahn5RyTnYEoAVsUjKgdKsg8oikWdiO8TAhTRsuh8CkfZaK1-ghG2M1REiHZW-vX1BlWD4zypZXcUaekAEK75b_YjQ7ho2-8g/file# [following]
--2023-06-14 07:02:26--  https://uc9f7f268526ae0b015cde1a709c.dl.dropboxusercontent.com/cd/0/inline/B9_rywAU_ogGIukQN1rdO3bB3SfEpTCnxV2z7FneG500DNRHDr7rEWnxSt-Lo3fXIoW6S

In [3]:
#!wget https://www.dropbox.com/s/d2wgk8j0wsye2ab/set_a.zip

In [4]:
#!unzip set_a.zip -d audio
!!tar -xvf genres.tar.gz?dl=0

['genres/',
 'genres/blues/',
 'genres/blues/blues.00000.au',
 'genres/blues/blues.00001.au',
 'genres/blues/blues.00002.au',
 'genres/blues/blues.00003.au',
 'genres/blues/blues.00004.au',
 'genres/blues/blues.00005.au',
 'genres/blues/blues.00006.au',
 'genres/blues/blues.00007.au',
 'genres/blues/blues.00008.au',
 'genres/blues/blues.00009.au',
 'genres/blues/blues.00010.au',
 'genres/blues/blues.00011.au',
 'genres/blues/blues.00012.au',
 'genres/blues/blues.00013.au',
 'genres/blues/blues.00014.au',
 'genres/blues/blues.00015.au',
 'genres/blues/blues.00016.au',
 'genres/blues/blues.00017.au',
 'genres/blues/blues.00018.au',
 'genres/blues/blues.00019.au',
 'genres/blues/blues.00020.au',
 'genres/blues/blues.00021.au',
 'genres/blues/blues.00022.au',
 'genres/blues/blues.00023.au',
 'genres/blues/blues.00024.au',
 'genres/blues/blues.00025.au',
 'genres/blues/blues.00026.au',
 'genres/blues/blues.00027.au',
 'genres/blues/blues.00028.au',
 'genres/blues/blues.00029.au',
 'genres/b

In [5]:
!rm genres.tar.gz?dl=0

In [6]:
import os
import pickle
from glob import iglob
import numpy as np
import librosa

DATA_AUDIO_DIR = './genres'
TARGET_SR = 8000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')
AUDIO_LENGTH = 10000

In [7]:
os.makedirs('/content/output/train/')
os.makedirs('/content/output/test/')

In [8]:
class_ids = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz':5,
    'metal':6,
    'pop':7,
    'reggae':8,
    'rock':9
}

audio_Class = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

def extract_class_id(wav_filename):
    if 'blues' in wav_filename:
        return class_ids.get('blues')
    elif 'classical' in wav_filename:
        return class_ids.get('classical')
    elif 'country' in wav_filename:
        return class_ids.get('country')
    elif 'disco' in wav_filename:
        return class_ids.get('disco')
    elif 'hiphop' in wav_filename:
        return class_ids.get('hiphop')
    elif 'jazz' in wav_filename:
        return class_ids.get('jazz')
    elif 'metal' in wav_filename:
        return class_ids.get('metal')
    elif 'pop' in wav_filename:
        return class_ids.get('pop')
    elif 'reggae' in wav_filename:
        return class_ids.get('reggae')
    else:
        return class_ids.get('rock')

In [9]:
extract_class_id('/content/genres/pop/pop.00000.au')

7

In [10]:
def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

In [11]:
def convert_data():
  for audioItem in audio_Class:
    path = DATA_AUDIO_DIR + '/' + audioItem
    #print(path)
    for i, wav_filename in enumerate(iglob(os.path.join(path, '**/**.au'), recursive=True)):
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            #print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            #print('CUT New length =', len(audio_buf))

        output_folder = OUTPUT_DIR_TRAIN
        if i // 10 == 0:
            output_folder = OUTPUT_DIR_TEST

        output_filename = os.path.join(output_folder, str(i) + audioItem + '.pkl')
        print(output_filename)
        out = {'class_id': class_id,
               'audio': audio_buf,
               'sr': TARGET_SR}
        w=open(output_filename,'wb')
        pickle.dump(out,w)
        w.close()



In [12]:
convert_data()

0 ./genres/blues/blues.00021.au 240107 -0.0 0.9999999
./output/test/0blues.pkl
1 ./genres/blues/blues.00026.au 240107 0.0 1.0
./output/test/1blues.pkl
2 ./genres/blues/blues.00048.au 240107 -0.0 1.0
./output/test/2blues.pkl
3 ./genres/blues/blues.00002.au 240107 0.0 1.0
./output/test/3blues.pkl
4 ./genres/blues/blues.00069.au 240107 0.0 0.99999994
./output/test/4blues.pkl
5 ./genres/blues/blues.00017.au 240107 0.0 1.0
./output/test/5blues.pkl
6 ./genres/blues/blues.00079.au 240107 -0.0 1.0
./output/test/6blues.pkl
7 ./genres/blues/blues.00015.au 240107 0.0 1.0
./output/test/7blues.pkl
8 ./genres/blues/blues.00042.au 240107 0.0 0.9999999
./output/test/8blues.pkl
9 ./genres/blues/blues.00096.au 240107 -0.0 1.0
./output/test/9blues.pkl
10 ./genres/blues/blues.00076.au 240107 -0.0 1.0
./output/train/10blues.pkl
11 ./genres/blues/blues.00014.au 240107 0.0 0.99999994
./output/train/11blues.pkl
12 ./genres/blues/blues.00097.au 240107 0.0 1.0
./output/train/12blues.pkl
13 ./genres/blues/blues.

In [13]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda,Conv1D, MaxPooling1D,Activation, Dense,BatchNormalization

from tensorflow.keras.models import Sequential
import numpy as np
import pickle
import os
from glob import glob

In [14]:
AUDIO_LENGTH = 10000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')

In [15]:
def m5(num_classes=5):
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [16]:
def get_data(file_list):
    def load_into(_filename, _x, _y):
        #print(_filename)
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)

In [17]:
#glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))

In [18]:
num_classes = len(audio_Class)
model = m5(num_classes=num_classes)

if model is None:
    exit('Something went wrong!!')

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Using Model M5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2500, 128)         10368     
                                                                 
 batch_normalization (BatchN  (None, 2500, 128)        512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 2500, 128)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 625, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 625, 128)          49280     
                                                                 
 batch_normalization_1 (Batc  (None, 625,

In [19]:
train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))
#train_files

In [20]:
x_tr, y_tr = get_data(train_files)

In [21]:
x_tr, x_tr.shape

(array([[[-0.2556215 ],
         [-0.3030297 ],
         [-0.96137387],
         ...,
         [-3.5525901 ],
         [-2.8898184 ],
         [-1.9176979 ]],
 
        [[-0.08155504],
         [-0.19547135],
         [-0.21467946],
         ...,
         [ 0.5317789 ],
         [-0.10659204],
         [-0.64437836]],
 
        [[ 0.63584816],
         [ 2.258963  ],
         [ 1.9075264 ],
         ...,
         [-0.01018783],
         [-0.01857104],
         [-0.34945825]],
 
        ...,
 
        [[-0.21481787],
         [-1.0771586 ],
         [-0.17563266],
         ...,
         [-0.4751068 ],
         [-0.3576234 ],
         [-0.37393615]],
 
        [[ 0.36336818],
         [-0.9366218 ],
         [-0.8664393 ],
         ...,
         [-0.2811306 ],
         [-0.10695738],
         [-0.11848165]],
 
        [[-0.2285599 ],
         [-0.15716277],
         [-0.25670153],
         ...,
         [-1.0509316 ],
         [-0.39651468],
         [ 0.44807094]]], dtype=float32),
 (90

In [22]:
y_tr, y_tr.shape

(array([8, 8, 5, 4, 6, 4, 6, 5, 1, 0, 0, 3, 9, 5, 6, 3, 9, 8, 6, 7, 4, 5,
        4, 9, 0, 6, 8, 7, 8, 1, 8, 9, 5, 1, 0, 3, 0, 1, 5, 1, 3, 3, 3, 2,
        0, 4, 0, 1, 8, 1, 9, 7, 1, 9, 1, 1, 7, 4, 9, 7, 9, 3, 9, 4, 9, 8,
        4, 6, 9, 7, 9, 7, 6, 2, 0, 7, 2, 1, 5, 5, 4, 9, 9, 0, 5, 2, 1, 9,
        4, 3, 6, 0, 6, 3, 0, 3, 7, 9, 3, 4, 8, 5, 8, 0, 1, 1, 5, 6, 1, 6,
        7, 5, 2, 0, 4, 3, 5, 4, 8, 6, 7, 1, 3, 6, 7, 4, 6, 8, 8, 6, 7, 0,
        4, 4, 6, 6, 4, 1, 7, 6, 6, 3, 3, 3, 3, 7, 1, 9, 3, 9, 4, 3, 7, 7,
        2, 8, 6, 6, 8, 7, 6, 8, 4, 6, 3, 5, 0, 0, 7, 3, 2, 1, 7, 9, 9, 5,
        5, 5, 2, 9, 4, 2, 7, 5, 5, 9, 6, 6, 0, 2, 9, 8, 7, 4, 1, 5, 8, 8,
        9, 2, 8, 9, 2, 3, 6, 7, 2, 9, 0, 4, 3, 4, 7, 6, 9, 5, 2, 5, 8, 3,
        4, 0, 1, 3, 7, 1, 7, 7, 5, 1, 6, 6, 5, 4, 4, 2, 8, 5, 9, 7, 4, 1,
        2, 6, 4, 1, 5, 2, 7, 9, 8, 4, 6, 1, 9, 9, 3, 4, 1, 2, 0, 9, 3, 8,
        3, 4, 1, 1, 2, 8, 1, 2, 7, 4, 3, 2, 4, 5, 3, 8, 3, 2, 6, 6, 3, 8,
        1, 2, 3, 2, 6, 0, 7, 6, 9, 0, 

In [23]:
y_tr = to_categorical(y_tr, num_classes=num_classes)

In [24]:
test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
x_te, y_te = get_data(test_files)

In [25]:
y_te = to_categorical(y_te, num_classes=num_classes)

In [26]:

print('x_tr.shape =', x_tr.shape)
print('y_tr.shape =', y_tr.shape)
print('x_te.shape =', x_te.shape)
print('y_te.shape =', y_te.shape)

x_tr.shape = (900, 10000, 1)
y_tr.shape = (900, 10)
x_te.shape = (100, 10000, 1)
y_te.shape = (100, 10)


In [27]:
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
batch_size = 128
model.fit(x=x_tr,
          y=y_tr,
          batch_size=batch_size,
          epochs=400,
          verbose=1,
          shuffle=True,
          validation_data=(x_te, y_te),
          callbacks=[reduce_lr])

Epoch 1/400
8/8 [==============================] - ETA: 0s - loss: 2.3448 - accuracy: 0.2367

8/8 [==============================] - 16s 170ms/step - loss: 2.3448 - accuracy: 0.2367 - val_loss: 2.3354 - val_accuracy: 0.1400 - lr: 0.0010
Epoch 2/400
7/8 [=========================>....] - ETA: 0s - loss: 1.8473 - accuracy: 0.3482

8/8 [==============================] - 1s 79ms/step - loss: 1.8462 - accuracy: 0.3489 - val_loss: 2.2825 - val_accuracy: 0.2300 - lr: 0.0010
Epoch 3/400
7/8 [=========================>....] - ETA: 0s - loss: 1.6774 - accuracy: 0.4252

8/8 [==============================] - 1s 78ms/step - loss: 1.6780 - accuracy: 0.4244 - val_loss: 2.2377 - val_accuracy: 0.2900 - lr: 0.0010
Epoch 4/400
7/8 [=========================>....] - ETA: 0s - loss: 1.5785 - accuracy: 0.4621

8/8 [==============================] - 1s 78ms/step - loss: 1.5826 - accuracy: 0.4611 - val_loss: 2.2740 - val_accuracy: 0.2400 - lr: 0.0010
Epoch 5/400
7/8 [=========================>....] - ETA: 0s - loss: 1.5190 - accuracy: 0.4766

...

8/8 [==============================] - 1s 86ms/step - loss: 0.1727 - accuracy: 0.9922 - val_loss: 3.2588 - val_accuracy: 0.4300 - lr: 0.0010
Epoch 397/400
7/8 [=========================>....] - ETA: 0s - loss: 0.1514 - accuracy: 0.9978

8/8 [==============================] - 1s 92ms/step - loss: 0.1513 - accuracy: 0.9978 - val_loss: 2.7480 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 398/400
7/8 [=========================>....] - ETA: 0s - loss: 0.1456 - accuracy: 0.9978

8/8 [==============================] - 1s 87ms/step - loss: 0.1461 - accuracy: 0.9978 - val_loss: 2.5622 - val_accuracy: 0.4700 - lr: 0.0010
Epoch 399/400
7/8 [=========================>....] - ETA: 0s - loss: 0.1467 - accuracy: 0.9978

8/8 [==============================] - 1s 86ms/step - loss: 0.1468 - accuracy: 0.9978 - val_loss: 2.3773 - val_accuracy: 0.4700 - lr: 0.0010
Epoch 400/400
7/8 [=========================>....] - ETA: 0s - loss: 0.1471 - accuracy: 0.9978

8/8 [==============================] - 1s 83ms/step - loss: 0.1471 - accuracy: 0.9978 - val_loss: 2.3221 - val_accuracy: 0.4600 - lr: 0.0010


In [46]:
predictions=model.predict(x_te)

4/4 [==============================] - 0s 8ms/step


In [47]:
predictions

array([[1.43573107e-02, 1.46449665e-02, 3.96443374e-06, 5.56626728e-05,
        1.27819134e-04, 9.68626082e-01, 9.08362665e-07, 1.07027299e-03,
        1.10580481e-03, 7.16533759e-06],
       [9.74256173e-03, 1.42028911e-09, 7.08318112e-06, 1.55157559e-05,
        5.74927306e-08, 5.43708506e-04, 2.56540843e-05, 4.44266061e-06,
        9.89640534e-01, 2.04500011e-05],
       [7.87907541e-01, 1.77093223e-03, 4.05543670e-02, 6.23829081e-04,
        4.67218133e-03, 8.09175521e-02, 2.47623375e-05, 2.08407786e-04,
        7.95836970e-02, 3.73662240e-03],
       [9.52642083e-01, 1.87910143e-02, 4.00674675e-04, 8.41720430e-06,
        3.53047450e-04, 1.61785427e-02, 3.37944834e-06, 9.34362039e-03,
        1.89512852e-03, 3.84128070e-04],
       [7.56297261e-03, 2.81517659e-05, 5.03386400e-06, 7.85766169e-04,
        9.67486918e-01, 3.34922248e-03, 1.36298031e-05, 3.49236047e-03,
        1.71048343e-02, 1.71085936e-04],
       [3.21367523e-04, 9.23193634e-01, 8.95162404e-04, 2.15056152e-05,
   

In [38]:
from sklearn.metrics import roc_auc_score

In [55]:
y_te.shape, predictions.shape

((1000,), (1000,))

In [48]:
roc_auc_score(y_te,predictions)

0.8805555555555555

In [49]:
predictions.shape[0], predictions.shape[1],

(100, 10)

In [50]:
for row in range(predictions.shape[0]):
  indexColumn = np.argmax(predictions[row])
  for column in range(predictions.shape[1]):
    if column == indexColumn:
      predictions[row][column] = 1
    else:
      predictions[row][column] = 0

In [52]:
predictions = predictions.reshape(-1)

In [53]:
y_te = y_te.reshape(-1)


In [54]:
roc_auc_score(y_te,predictions)

0.7